<div style="text-align: right;">
  <img src="https://raw.githubusercontent.com/exasol/ai-lab/refs/heads/main/assets/Exasol_Logo_2025_Dark.svg" style="width:200px; margin: 10px;" />
</div>

# Advanced topics

This notebooks explains some details and background regarding the tool `exaslct`. This is especially useful when:
- you encounter a problem when running one of the other notebooks in this tutorial.
- you need to clean up disk space.
- you want to do more modifications to the script-languages-container than just adding a Python package.

## Setup
### Open Secure Configuration Storage

In [ ]:
%run ../utils/access_store_ui.ipynb
display(get_access_store_ui('../'))

### Instantiate ScriptLanguagesContainer

The following cell creates an instance of class `ScriptLanguageContainer` from the notebook-connector,
which enables using the`exaslct` in the AI Lab in a convenient way.

In [ ]:
from exasol.nb_connector.slc import ScriptLanguageContainer
slc = ScriptLanguageContainer(secrets=ai_lab_config, name="sample_slc")

### Import some utility functions


In [ ]:
%run ./utils/file_system_ui.ipynb

### Preparation 
Before you start, run the export command (again), just to be sure to have all the artifacts (local files and docker images), which are necessary for this tutorial.

In [ ]:
slc.export()

## What to do if something doesn't work?

During the build, export or upload it can happen that external package repositories are not available or something is wrong on your machine running the build. For these cases, `exaslct` provides extensive log files that can help analyzing such problems.

#### Exaslsct Log
The main log file for `exaslct` is stored as file `main.log` in the build output of the job. With the following command you can find the main logs for all previous executions.

In [ ]:
main_logs = list(slc.workspace.output_path.glob('**/main.log'))
show_files(main_logs)

The following command shows the log file of the last execution.

In [ ]:
tail_file(main_logs[0], 20)

#### Build Output Directory

More detailed information about the build or other operations can be found in directory `.build_output/jobs/*/outputs`. Here each run of `exaslct` creates its own subdirectory under `.build_output/jobs`. Directory `outputs`  contains the outputs and log files (if any) produced by each of the executed tasks of `exaslct`. Especially, the Docker tasks such as build, pull and push store the logs returned by the Docker API. This can be helpful for analyzing problems during the build.

In [ ]:
all_logs = list(slc.workspace.output_path.glob('**/*.log'))
show_files(all_logs)

<a id='flavor_definition'></a>
## Flavor Definition
The following diagram shows a high level overview of the build steps for a script languages container.

Building an SLC usually starts with selecting one of the default build templates provided by Exsol.
These templates are called _flavors_. In this tutorial we customize the template Python flavor by adding new pip packages.


![image.png](slc_main_build_steps.svg)





The easiest way to customize a flavor is to add dependencies in the build step `flavor_customization`. Other build steps should only be changed with caution and is only required in special cases, e.g. when dependencies are defined in other build steps because the script client depends on these dependencies.
Check the directory structure of the selected flavor:


In [ ]:
show_directory_content(slc.flavor_path, 3)

For example, if you need additional apt packages, you can add those to the `template-Exasol-all-python-3.10/flavor_customization/packages/apt_get_packages` file.

## Build Cache

`exaslct` internally uses a build cache in order to accelerate the build by re-using docker images, which were built during a previous execution.
Customizing a flavor always creates a separate entry in the cache with a unique hashcode and old containers don't get lost. If you revert your changes the system automatically uses the existing cached container. Below you can see the content of the cache directory for the containers.

In [ ]:
show_directory_content(slc.workspace.output_path / "cache" / "exports")

`exaslct` also creates a docker image for each particular build step (see [Flavor Definition](#flavor_definition)), you can find the images with the following code:

In [ ]:
slc.docker_image_tags

Image `exasol/script-language-container:template-Exasol-all-python-3.10-release_...` is the final release image for the script-language-container, the other (intermediate) images are used for the caching mechnism.

## Cleanup

This sections shows how you can use the instance of `ScriptLanguageContainer` class to clean up the artifacts created by `exaslct`:
- The docker images for the script-languages-container as well as the cache images
- The exported containers (tar gz files)
- The cached container files


### Clean up the docker images

The following command cleans up the docker images.
**Be aware that this command deletes all script-languages-container related docker images on the running host, even those created by other notebooks or users.**

In [ ]:
slc.clean_docker_images()

### Clean up the local exported containers

The following command cleans up the exported container files:

In [ ]:
slc.workspace.cleanup_export_path()

### Clean up the output path

Clean up the log files and caches:

In [ ]:
slc.workspace.cleanup_output_path()